In [1]:
import pyodbc
import pandas as pd
import os
import datetime as dt
from datetime import datetime, timedelta, date
import xlrd
import pytz
import import_ipynb
from d0_sql_connection import DataFromSQL
from d0_sql_connection import *


In [2]:
#load the location variables from the file
with open('location_variables.txt', 'r') as f:
    for line in f:
        exec(line)

In [3]:
run_time_naive

'2025-04-10 13:03:07'

In [4]:
run_date

'2025-04-10 00:00:00'

In [5]:

# List of queries with corresponding output file names
queries = {
    "stock": """
        WITH MaxDate AS (
        SELECT 
        material_sk, 
        plant_sk, 
        storage_location_code, 
        batch_code,
        MAX(period_start_ts) AS max_period_start_ts,
        period_end_ts
        FROM abi_logistics_europe_non_sensitive_rpt.batch_stock_eur_d_v
        group by material_sk, plant_sk, storage_location_code, batch_code, period_end_ts)
        
        SELECT m.material_code, s.*
        FROM abi_logistics_europe_non_sensitive_rpt.batch_stock_eur_d_v AS s
        LEFT JOIN abi_logistics_dwh.material_eur_d AS m
        ON s.material_sk = m.material_sk
        LEFT JOIN maxdate as mx
        on mx.material_sk = s.material_sk AND mx.plant_sk = s.plant_sk AND mx.storage_location_code = s.storage_location_code AND mx.batch_code = s.batch_code AND mx.max_period_start_ts = s.period_start_ts
        WHERE m.src_sys_region_nm = 'europe'
        and s.valuated_stock_hl >0
        order by s.period_start_ts desc
    """,
    "Planned_loads": """
        WITH Stop_1_Filtered AS (
        SELECT distinct load_leg_sk, shipping_location_sk, appointment_from_ts, appointment_to_ts
        FROM abi_logistics_dwh.stop_hist_actual_eur_d
        WHERE period_end_ts > GETDATE()
    ),
    Stop_2_Filtered AS (
        SELECT distinct load_leg_sk, shipping_location_sk, appointment_from_ts, appointment_to_ts
        FROM abi_logistics_dwh.stop_hist_actual_eur_d
        WHERE period_end_ts > GETDATE()
    )
    SELECT *,
        dh.load_leg_item_number AS load_id,
        stop_1.appointment_from_ts AS origin_slot_arrival,
        stop_1.appointment_to_ts AS origin_slot_departure,
        stop_2.appointment_from_ts AS destination_slot_arrival,
        stop_2.appointment_to_ts AS destination_slot_departure,
        di.reference_document_number AS RFRC_NUM12,
        osl.shipping_location_code AS origin,
        dsl.shipping_location_code AS destination,
        di2.distribution_channel_code,  -- Use distribution_channel_code from the subquery
        CASE
            WHEN di2.distribution_channel_code = '01' THEN 'SO'
            WHEN di2.distribution_channel_code = '02' THEN 'STO'
            WHEN di2.distribution_channel_code = '03' THEN 'Export'
            ELSE 'Undefined'
        END AS movement_type
    FROM abi_logistics_dwh.delivery_item_eur_d di
    LEFT JOIN abi_logistics_dwh.delivery_eur_d dh
        ON di.delivery_sk = dh.delivery_sk
    LEFT JOIN abi_logistics_dwh.material_eur_d mat
        ON di.material_sk = mat.material_sk
    LEFT JOIN abi_logistics_dwh.shipping_location_eur_d osl
        ON dh.origin_shipping_location_sk = osl.shipping_location_sk
    LEFT JOIN abi_logistics_dwh.shipping_location_eur_d dsl
        ON dh.destination_shipping_location_sk = dsl.shipping_location_sk
    LEFT JOIN Stop_1_Filtered stop_1
        ON dh.load_leg_sk = stop_1.load_leg_sk
        AND dh.origin_shipping_location_sk = stop_1.shipping_location_sk
    LEFT JOIN Stop_2_Filtered stop_2
        ON dh.load_leg_sk = stop_2.load_leg_sk
        AND dh.destination_shipping_location_sk = stop_2.shipping_location_sk
    LEFT JOIN (
        SELECT
            delivery_sk,
            plant_code,
            MAX(distribution_channel_code) AS distribution_channel_code
        FROM abi_logistics_dwh.delivery_item_eur_d
        WHERE act_flg= 'Y'
        AND actual_quantity > 0
        GROUP BY delivery_sk, plant_code
    ) di2
    ON dh.delivery_sk = di2.delivery_sk
    WHERE osl.shipping_location_code LIKE 'GB%'
    AND di.actual_quantity > 0
    AND di.act_flg = 'Y'
    AND stop_1.appointment_from_ts IS NOT NULL
    AND stop_1.appointment_to_ts IS NOT NULL
    AND (
        CAST(stop_1.appointment_from_ts AS DATE) = CAST(GETDATE() AS DATE)
        OR CAST(stop_1.appointment_from_ts AS DATE) = CAST(DATEADD(DAY, 1, GETDATE()) AS DATE)
    )
    AND dh.delivery_sales_document_category <> '7';
    """,
    "Inventory_policy": """
        WITH LatestUpload AS (
            SELECT *,
                ROW_NUMBER() OVER (PARTITION BY material_sk, plant_sk , recommendation_date
                                    ORDER BY ins_gmt_ts DESC, upd_gmt_ts DESC) AS rn
            FROM abi_logistics_dwh.planning_inventory_profile_eur_f
            WHERE LEN(plant_code) = 4 
            AND recommendation_date = CAST(GETDATE() AS DATE)
            AND (target_opening_stock != 0 OR maximum_stock != 0)
            AND (business_unit_code = 'BNFL' OR business_unit_code = 'Central'
                OR business_unit_code = 'West' OR business_unit_code = 'East')
        )
        SELECT *
        FROM LatestUpload
        WHERE rn = 1;
    """,
    "Planned_production": """
        SELECT *
        FROM abi_logistics_dwh.infor_pub_process_batch_hist_eur_f
        WHERE period_end_ts = (
            SELECT MAX(period_end_ts)
            FROM abi_logistics_dwh.infor_pub_process_batch_hist_eur_f
        )
        AND (
            (CAST(start_inflow_ts AS DATE) <= CAST(GETDATE() AS DATE) 
            AND CAST(end_outflow_ts AS DATE) >= CAST(GETDATE() AS DATE))
            OR 
            (CAST(start_inflow_ts AS DATE) <= CAST(DATEADD(DAY, 1, GETDATE()) AS DATE) 
            AND CAST(end_outflow_ts AS DATE) >= CAST(DATEADD(DAY, 1, GETDATE()) AS DATE))
        );
    """,
    "Actual_production": """
        SELECT *
        FROM abi_logistics_europe_non_sensitive_rpt.fpa_sigma_actual_v
        WHERE reporting_date = CAST(GETDATE() AS DATE);
    """,
    "UOM_full": """
        SELECT m.material_code, uom.* 
        FROM abi_logistics_dwh.material_measure_conversion_eur_d AS uom
        LEFT JOIN abi_logistics_dwh.material_eur_d AS m
        ON uom.material_sk = m.material_sk
        WHERE uom.target_unit_of_measure_code IN ('HL', 'PAL') 
        AND m.src_sys_region_nm = 'europe';
    """,
    "UOM_weight": """
        SELECT DISTINCT
            did.material_code,
            did.material_description,
            hier.container_type_description,
            case_length,
            case_width,
            case_height,
            case_weight_kg,
            layer_length,
            layer_width,
            layer_height,
            layer_weight_kg,
            cha_pal.length 'pal_length',
            cha_pal.width 'pal_width',
            pal_height,
            CASE
                WHEN cha_pal.pal_weight_kg IS NOT NULL
                    THEN cha_pal.pal_weight_kg
                ELSE cha_pal.gross_weight
            END 'pal_weight_kg',
            case_to_pal,
            case_to_layer,
            layer_to_pal
        FROM (
            SELECT DISTINCT 
                mara.material_sk,
                mara.material_code,
                mara.material_description,
                mara.product_hierarchy_sk
            FROM abi_logistics_dwh.material_eur_d mara 
            WHERE mara.src_sys_id = '615' 
                AND base_unit_of_measure_code NOT IN ('L', 'HL')
        ) did
        LEFT JOIN abi_logistics_dwh.product_hierarchy_flattened_eur_d hier
        ON did.product_hierarchy_sk = hier.product_hierarchy_sk
        LEFT JOIN (
            SELECT
                mat.material_sk,
                1* uom.conversion_numerator / uom.conversion_denumerator * pc_mMC.conversion_denumerator / pc_mMC.conversion_numerator 'case_to_pal',
                1* lay_mMC.conversion_numerator / lay_mMC.conversion_denumerator * pc_mMC.conversion_denumerator / pc_mMC.conversion_numerator 'case_to_layer',
                1* uom.conversion_numerator / uom.conversion_denumerator * lay_mMC.conversion_denumerator / lay_mMC.conversion_numerator 'layer_to_pal',
                cha.internal_floating_point_from_value 'pal_weight_kg',
                CASE 
                    WHEN cha_pal_h.internal_floating_point_from_value IS NOT NULL
                        THEN cha_pal_h.internal_floating_point_from_value
                    ELSE uom.height
                END 'pal_height',
                uom.length,
                uom.width,
                uom.gross_weight,
                pc_mMC.length 'case_length',
                pc_mMC.width 'case_width',
                pc_mMC.height 'case_height',
                pc_mMC.gross_weight 'case_weight_kg',
                lay_mMC.length 'layer_length',
                lay_mMC.width 'layer_width',
                lay_mMC.height 'layer_height',
                lay_mMC.gross_weight 'layer_weight_kg'
            FROM abi_logistics_dwh.material_eur_d mat
            LEFT JOIN abi_logistics_dwh.material_characteristic_eur_d cha
            ON mat.material_sk = cha.material_sk
            AND cha.internal_characteristic_code = '778'
            LEFT JOIN abi_logistics_dwh.material_characteristic_eur_d cha_pal_h
            ON mat.material_sk = cha_pal_h.material_sk
            AND cha_pal_h.internal_characteristic_code = '785'
            LEFT JOIN abi_logistics_dwh.material_measure_conversion_eur_d uom
            ON mat.material_sk = uom.material_sk
            AND target_unit_of_measure_code = 'PAL'
            LEFT JOIN abi_logistics_dwh.material_measure_conversion_eur_d pc_mMC
            ON pc_mMC.target_unit_of_measure_code = 'PC'
            AND pc_mMC.material_sk = mat.material_sk
            LEFT JOIN abi_logistics_dwh.material_measure_conversion_eur_d lay_mMC
            ON lay_mMC.target_unit_of_measure_code = 'LAYER'
            AND lay_mMC.material_sk = mat.material_sk
            LEFT JOIN abi_logistics_dwh.material_measure_conversion_eur_d cha_pal
            ON cha_pal.material_sk = mat.material_sk
            AND cha_pal.target_unit_of_measure_code = 'PAL'
            WHERE mat.src_sys_region_nm = 'europe'
        ) cha_pal
        ON did.material_sk = cha_pal.material_sk
        WHERE cha_pal.pal_weight_kg IS NOT NULL;
    """,
    "lcp_data": """
        SELECT * from abi_logistics_dwh.lcp_rank_operational_forecast_eur_f
        WHERE LEN(origin_location_code) = 4
        AND LEN(destination_location_code) = 4;
    """
}


In [6]:
# Function to execute query and save result as a CSV file using DataFromSQL class
def execute_and_save(query, data_filepath, filename, sql_obj, conxn):
    status, df = sql_obj.load_sql_data(query, conxn)
    if status:
        output_path = os.path.join(data_filepath, f"input_{filename}.csv")
        df.to_csv(output_path,index=False)
        print(f"Saved {filename} as CSV at: {output_path}")
    else:
        print(f"Failed to execute query for {filename}")

# Initialize SQL connection using DataFromSQL class
try:
    sql_obj = DataFromSQL()
    connection_status, conxn = sql_obj.sql_server_setup(Server, DataBase, Username, Password)
    print(f"SQL Server Connection Status is: {connection_status}")

    # Loop through the queries and execute each one
    for name, query in queries.items():
        execute_and_save(query, data_filepath, name, sql_obj, conxn)

    # Close the connection
    conxn.close()

except Exception as e:
    raise Exception(str(e))

SQL Server Connection Status is: Successfull


<string>:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


Saved stock as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_stock.csv


Saved Planned_loads as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_Planned_loads.csv


Saved Inventory_policy as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_Inventory_policy.csv


Saved Planned_production as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_Planned_production.csv


Saved Actual_production as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_Actual_production.csv


Saved UOM_full as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_UOM_full.csv


Saved UOM_weight as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_UOM_weight.csv


Saved lcp_data as CSV at: ./Input_Data/04. April 2025/10_04_2025/2025_04_10_13_03_07 input data/input_lcp_data.csv
